In [1]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

In [2]:
# import os
# os.chdir('/kaggle/input/test-autoannotation')

# **Install**

In [3]:
!pip install ultralytics


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.0/869.0 kB 14.9 MB/s eta 0:00:00a 0:00:01


# **Imports**

In [4]:
# Import necessary libraries
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from ultralytics import YOLO
from tqdm.notebook import tqdm
import shutil

# **Data Ingestion**

In [5]:
def load_data(data_dir):
    # Define paths to dataset
    train_images_path = os.path.join(data_dir, 'train', '*.jpg')
    val_images_path = os.path.join(data_dir, 'val', '*.jpg')
    # test_images_path = os.path.join(data_dir, 'test', '*.jpg')

    # Get list of image files
    train_images = glob(train_images_path)
    val_images = glob(val_images_path)
    # test_images = glob(test_images_path)

    return train_images, val_images

In [6]:
data_dir = '/kaggle/input/test-autoannotation/datset_yolo_annotation/coco_datset'  # Update with your dataset directory
train_dir ='/kaggle/input/test-autoannotation/datset_yolo_annotation/coco_datset/train'
val_dir = '/kaggle/input/test-autoannotation/datset_yolo_annotation/coco_datset/val'
test_dir = '/kaggle/input/test-autoannotation/datset_yolo_annotation/coco_datset/test'
train_images, val_images = load_data(data_dir)

# **Model Loading**

In [7]:
os.chdir('/kaggle/working/')

MODEL = "yolov8x.pt"

def load_model():
    model = YOLO(MODEL)
    model.fuse()   # Fuse method is called here
    return model

# Load and fuse the model
model = load_model()

100%|██████████| 131M/131M [00:00<00:00, 287MB/s] 


YOLOv8x summary (fused): 268 layers, 68,200,608 parameters, 0 gradients, 257.8 GFLOPs


# **Preprocessing**

In [8]:
# !pip install pycocotools

In [9]:
# def preprocess_image(image_path):
#     image = cv2.imread(image_path)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     return image


# def preprocess_annotations(annotation_path):
#     # Function to parse annotations
#     with open(annotation_path, 'r') as file:
#         annotations = file.readlines()
#     return annotations

In [10]:
# import numpy as np
# from pycocotools.coco import COCO

# def preprocess_image(image_path):
#     # Custom image preprocessing logic here
#     image = cv2.imread(image_path)  # Example using OpenCV
#     image = cv2.resize(image, (640, 640))  # Example resizing
#     return image

# def preprocess_data(images, coco):
#     preprocessed_images = []
#     annotations = []
#     for image_path in tqdm(images):
#         image = preprocess_image(image_path)

#         # Get image ID from the filename
#         image_id = int(os.path.basename(image_path).replace('.jpg', ''))

#         # Get corresponding annotation for this image from COCO
#         ann_ids = coco.getAnnIds(imgIds=[image_id])
#         anns = coco.loadAnns(ann_ids)

#         preprocessed_images.append(image_path)  # Store the path instead of the image
#         annotations.append(anns)
#     return preprocessed_images, annotations

# # Paths to the annotations JSON files
# train_annotations_file = '/kaggle/input/test-autoannotation/annotations/annotations/train_coco_annotations.json'
# val_annotations_file = '/kaggle/input/test-autoannotation/annotations/annotations/val_coco_annotations.json'

# # Load COCO annotation files
# train_coco = COCO(train_annotations_file)
# val_coco = COCO(val_annotations_file)

# # Preprocess data
# train_images_preprocessed, train_annotations = preprocess_data(train_images, train_coco)
# val_images_preprocessed, val_annotations = preprocess_data(val_images, val_coco)


In [11]:
# # Print basic information
# print("Training COCO Data:")
# print("Number of images:", len(train_coco.getImgIds()))
# print("Number of annotations:", len(train_coco.getAnnIds()))
# print("Categories:", train_coco.loadCats(train_coco.getCatIds()))

# print("\nValidation COCO Data:")
# print("Number of images:", len(val_coco.getImgIds()))
# print("Number of annotations:", len(val_coco.getAnnIds()))
# print("Categories:", val_coco.loadCats(val_coco.getCatIds()))

# **Training & Evaluation**

In [12]:
import wandb
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [13]:
!yolo --version

WARNING ⚠️ argument '--version' does not require leading dashes '--', updating to 'version'.
8.2.78


In [14]:
# import os
# import yaml

# # Change directory to where the YAML file is located
# # os.chdir('/kaggle/input/test-autoannotation')

# yaml_data_path = '/kaggle/input/test-autoannotation/data_coco.yaml'

# def train_model(model, train_images, train_annotations, val_images, val_annotations):
#     # Load the YAML file and check for required keys
#     with open(yaml_data_path, 'r') as f:
#         data = yaml.safe_load(f)

#     if 'names' not in data:
#         raise ValueError("The 'names' key is missing from your YAML file.")
#     if 'nc' not in data:
#         raise ValueError("The 'nc' key is missing from your YAML file.")

#     # Use the YAML file path directly
#     model.train(data=yaml_data_path)  # Pass the path to the YAML file

# # Train the model
# train_model(model, train_images_preprocessed, train_annotations, val_images_preprocessed, val_annotations)

In [17]:
import os
import yaml

yaml_data_path = '/kaggle/input/test-autoannotation/data_coco.yaml'

def train_model(model):
    # Load the YAML file and check for required keys
    with open(yaml_data_path, 'r') as f:
        data = yaml.safe_load(f)

    if 'names' not in data:
        raise ValueError("The 'names' key is missing from your YAML file.")
    if 'nc' not in data:
        raise ValueError("The 'nc' key is missing from your YAML file.")

    # Use the YAML file path directly
    model.train(data=yaml_data_path, epochs=200, imgsz=224, plots=True)  # Pass the path to the YAML file

# Train the model
train_model(model)

Ultralytics YOLOv8.2.78 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=/kaggle/input/test-autoannotation/data_coco.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_label

100%|██████████| 755k/755k [00:00<00:00, 12.3MB/s]
2024-08-18 14:42:58,984	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-08-18 14:42:59,793	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Overriding model.yaml nc=80 with nc=298

                   from  n    params  module                                       arguments                     
  0                  -1  1      2320  ultralytics.nn.modules.conv.Conv             [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.block.C2f             [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.block.C2f             [320, 320, 6, True]           
  5                  -1  1   1844480  ultralytics.nn.modules.conv.Conv             [320, 640, 3, 2]              
  6                  -1  6  13117440  ultralytics.nn.modules.block.C2f             [640, 640, 6, True]           
  7                  -1  1   3687680  ultralyti

wandb: Currently logged in as: himanko112 (himankoboruah). Use `wandb login --relogin` to force relogin


Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...


...


141 epochs completed in 0.707 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 137.3MB
Optimizer stripped from runs/detect/train/weights/best.pt, 137.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.78 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Model summary (fused): 268 layers, 68,410,542 parameters, 0 gradients, 259.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:05<00:00,  1.37s/it]


                   all        101        905    0.00513     0.0667    0.00647    0.00567
                diaper          1          1          0          0          0          0
            dog collar          2          2          0          0          0          0
             reflector         54        193     0.0102    0.00518    0.00519    0.00363
               glasses          4          9    0.00562      0.111    0.00328   0.000656
              umbrella          2          3          0          0          0          0
         traffic light          4          7          0          0          0          0
                   tag          1          1          0          0          0          0
              suitcase          1          1          0          0          0          0
            skateboard          2          2          0          0          0          0
power plugs and sockets          3          3          0          0          0          0
               hot d

wandb: Network error resolved after 0:00:01.193429, resuming normal operation.


lr/pg0,▅███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
lr/pg1,▅███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
lr/pg2,▅███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
metrics/mAP50(B),▁▂▂█▇▆▆▅▆▆▅▅██▆▅▅▅▅▅▅▄▄▄▄▄▄▄▃▄▃▃▃▃▄▃▃▃▃▃
metrics/mAP50-95(B),▁▁▂█▆▆▅▅▅▅▅▅█▇▆▅▅▅▅▄▅▄▄▄▄▃▃▃▃▃▃▃▃▃▄▃▃▃▃▃
metrics/precision(B),▁▂▂█▃▃▂▃▃▃▃▃▅▅▃▃▃▃▃▃▃▂▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
metrics/recall(B),▁▁▂▂▄▅▆▆▇███▇▇████▇█▇▇▆▇▇▇▇▇▆▆▆▆▅▅▆▅▆▆▆▅
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▆▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


In [19]:
data_dir = '/kaggle/input/test-autoannotation/datset_yolo_annotation/coco_datset'  # Update with your dataset directory
test_dir = '/kaggle/input/test-autoannotation/datset_yolo_annotation/coco_datset/test'
train_images, val_images = load_data(data_dir)

def load_test_data(data_dir):
    test_images_path = os.path.join(data_dir, 'test', '*.jpg')
    test_images = glob(test_images_path)

    return test_images

test_images = load_test_data(data_dir)

In [27]:
!yolo task=detect mode=predict model=/kaggle/working/yolov8x.pt source=/kaggle/input/test-autoannotation/data/data/test

Ultralytics YOLOv8.2.78 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLOv8x summary (fused): 268 layers, 68,200,608 parameters, 0 gradients, 257.8 GFLOPs

image 1/100 /kaggle/input/test-autoannotation/data/data/test/000000000139.jpg: 448x640 2 persons, 3 chairs, 2 potted plants, 2 dining tables, 2 tvs, 1 microwave, 2 refrigerators, 1 clock, 4 vases, 77.6ms
image 2/100 /kaggle/input/test-autoannotation/data/data/test/000000000285.jpg: 640x608 1 bear, 78.7ms
image 3/100 /kaggle/input/test-autoannotation/data/data/test/000000000632.jpg: 512x640 1 bottle, 1 chair, 2 potted plants, 1 bed, 2 books, 83.9ms
image 4/100 /kaggle/input/test-autoannotation/data/data/test/000000000724.jpg: 640x480 1 car, 2 stop signs, 72.6ms
image 5/100 /kaggle/input/test-autoannotation/data/data/test/000000000776.jpg: 640x448 1 couch, 3 teddy bears, 72.1ms
image 6/100 /kaggle/input/test-autoannotation/data/data/test/000000000785.jpg: 448x640 1 person, 2 skiss, 31.6ms
image 7/100 /kaggle/in

In [31]:
# import os

# def print_evaluation_metrics(log_dir):
#     log_files = glob(os.path.join(log_dir, '*.jpg'))
#     for log_file in log_files:
#         with open(log_file, 'r') as f:
#             print(f.read())

# # Specify the directory where evaluation logs are saved
# log_dir = '/kaggle/working/runs/detect/predict/'  # Update this based on your setup

# print_evaluation_metrics(log_dir)
